
# Graph Optimization

This example shows how to optimize a graph using pattern optimization.
The graph was obtained by running a dummy llama model.
It is the backward graph.

## A model


In [ ]:
import os
import onnx
import pandas
from onnx_array_api.plotting.text_plot import onnx_simple_text_plot
from onnx_array_api.plotting.graphviz_helper import plot_dot
from experimental_experiment.xbuilder.graph_builder import (
    GraphBuilder,
    OptimizationOptions,
)

filename = (
    os.path.join(os.path.dirname(__file__), "data", "dort-c-custom__1.onnx")
    if "__file__" in globals()
    else "data/dort-c-custom__1.onnx"
)
proto = onnx.load(filename)

print(f"number of nodes: {len(proto.graph.node)}")


print(onnx_simple_text_plot(proto))

And visually.



In [ ]:
plot_dot(proto)

## Optimization



In [ ]:
gr = GraphBuilder(
    proto,
    infer_shapes=True,
    optimization_options=OptimizationOptions(patterns="default"),
)
stats = gr.optimize()
df = pandas.DataFrame(stats)
df.to_csv("plot_optimize.csv")
df.to_excel("plot_optimize.xlsx")
df

Summary



In [ ]:
print(df[["pattern", "added", "removed"]].groupby("pattern").sum())

The total is:



In [ ]:
diff = df["added"].sum() - df["removed"].sum()

print(f"number of removed nodes: {-diff}")

Conversion to onnx.



In [ ]:
optimized_proto = gr.to_onnx(optimize=False)

print(f"number of new nodes: {len(optimized_proto.graph.node)}")

It gives the following.



In [ ]:
print(onnx_simple_text_plot(optimized_proto))

And visually.



In [ ]:
plot_dot(optimized_proto)

The first list of patterns optimizes the graph with only
standard onnx operators: `l-pattern-optimization-onnx`.
The second list is specific to :pekg:`onnxruntime`:
`l-pattern-optimization-ort`.

